In [43]:
import osmnx as ox
import matplotlib.pyplot as plt
import json
import elevation as dem
import rasterio as rio
from rasterio.warp import Resampling

# Install gdal according to python version: GDAL-3.4.2-cp39-cp39-win_amd64.whl
# Install fiona according to python version: https://www.lfd.uci.edu/~gohlke/pythonlibs/#fiona
import geopandas as gpd
import pandas as pd
import numpy as np

# turn response caching off/on
# ox.settings.use_cache = False

In [129]:
roi = (49.81061769757704, 9.988233595428222)
roi = (51.8903, 10.41933)
gdf = ox.geometries_from_point(roi, dist=200, tags={'building': True})
with open('./roi.json', 'w', encoding='utf-8') as f:
    formatted_data = json.loads(gdf[['geometry', 'description', 'name']].to_json())
    json.dump(formatted_data, f, ensure_ascii=False, indent=4)

# gdf

In [107]:
roi = (51.8903, 10.41933) # Rammelsberg Coordinates (Lat, Lng)
fname = f"{os.getcwd()}/Rammelsberg-DEM.tif"
print(fname)

offset = 0.01 # is it 100m!
bbx = (roi[1]-offset, roi[0]-offset, roi[1]+offset, roi[0]+offset)
dem.clip(bounds=(bbx[0], bbx[1], bbx[2], bbx[3]), output=fname, product='SRTM1')
dem.clean() # clean up stale temporary files and fix the cache in the event of a server error
dem_data = rio.open(fname)

# resample data
upscale_factor = 30
data = dem_data.read(1,
    out_shape=( int(dem_data.height * upscale_factor), int(dem_data.width * upscale_factor)),
    out_dtype=np.double,
    resampling=Resampling.bilinear
)

# scale image transform
transform = dem_data.transform * dem_data.transform.scale(
    (dem_data.width / data.shape[-1]),
    (dem_data.height / data.shape[-2])
)


/home/ahmad/personal_ws/src/ROSExamples/script/Rammelsberg-DEM.tif


In [138]:
json_data = None
with open("./roi.json", "r") as f:
    json_data = json.load(f) # load as dictionary
    for feat in json_data['features']:
        for pt in feat['geometry']['coordinates'][0]:
            if len(pt) < 3:
                elev = data[rio.transform.rowcol(transform, pt[0], pt[1])] # lng, lat
                pt.append(elev)
            print(pt)

with open('./roi.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

[10.4194923, 51.8905509, 332.0]
[10.4195133, 51.8905535, 333.0]
[10.4195576, 51.8905588, 334.0]
[10.419623, 51.8905666, 336.0]
[10.4200098, 51.8906128, 345.0]
[10.4200329, 51.8905392, 346.0]
[10.4202259, 51.8905623, 352.0]
[10.4202393, 51.8905194, 352.0]
[10.42055, 51.8905565, 361.0]
[10.4205732, 51.8904826, 361.0]
[10.4206119, 51.8903593, 362.0]
[10.420635, 51.8902857, 362.0]
[10.4203243, 51.8902486, 352.0]
[10.4203385, 51.8902033, 352.0]
[10.4203267, 51.8902019, 352.0]
[10.4203706, 51.8900617, 353.0]
[10.4199722, 51.890014, 338.0]
[10.4200135, 51.8898822, 340.0]
[10.4199582, 51.8898755, 339.0]
[10.419973, 51.8898284, 339.0]
[10.4199758, 51.8898193, 339.0]
[10.419984, 51.8897933, 339.0]
[10.4198617, 51.8897787, 337.0]
[10.4197493, 51.8897653, 334.0]
[10.4196888, 51.8897581, 333.0]
[10.419669, 51.8898212, 333.0]
[10.4194517, 51.8905461, 331.0]
[10.4194923, 51.8905509, 332.0]
[10.4193428, 51.8910365, 330.0]
[10.419271, 51.8912896, 328.0]
[10.4194709, 51.8913114, 331.0]
[10.4194602, 51.8